In [64]:
import mysql.connector

# Replace with your MySQL username and password
config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Srisql@8'
}

# Connect to MySQL server
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# Create database if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS food_wastage")

# Use the created database
cursor.execute("USE food_wastage")

print("Database 'food_wastage' is ready to use!")


Database 'food_wastage' is ready to use!


In [65]:
# Drop tables in correct order to avoid foreign key issues
cursor.execute("DROP TABLE IF EXISTS Claims")
cursor.execute("DROP TABLE IF EXISTS Food_Listings")
cursor.execute("DROP TABLE IF EXISTS Receivers")
cursor.execute("DROP TABLE IF EXISTS Providers")
conn.commit()

# Create tables
cursor.execute("""
CREATE TABLE Providers (
    Provider_ID INT PRIMARY KEY,
    Name VARCHAR(255),
    Type VARCHAR(100),
    Address VARCHAR(255),
    City VARCHAR(100),
    Contact VARCHAR(100)
)
""")

cursor.execute("""
CREATE TABLE Receivers (
    Receiver_ID INT PRIMARY KEY,
    Name VARCHAR(255),
    Type VARCHAR(100),
    City VARCHAR(100),
    Contact VARCHAR(100)
)
""")

cursor.execute("""
CREATE TABLE Food_Listings (
    Food_ID INT PRIMARY KEY,
    Food_Name VARCHAR(255),
    Quantity INT,
    Expiry_Date DATE,
    Provider_ID INT,
    Provider_Type VARCHAR(100),
    Location VARCHAR(100),
    Food_Type VARCHAR(100),
    Meal_Type VARCHAR(100),
    FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID)
)
""")

cursor.execute("""
CREATE TABLE Claims (
    Claim_ID INT PRIMARY KEY,
    Food_ID INT,
    Receiver_ID INT,
    Status VARCHAR(50),
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID)
)
""")

conn.commit()

print("Tables created successfully!")


Tables created successfully!


In [66]:
import pandas as pd

# Load CSV files - use raw string (r'path') to avoid path issues
providers_df = pd.read_csv(r'C:\Labmentix\Food waste Management\providers_data.csv')
receivers_df = pd.read_csv(r'C:\Labmentix\Food waste Management\receivers_data.csv')
food_listings_df = pd.read_csv(r'C:\Labmentix\Food waste Management\food_listings_data.csv')
claims_df = pd.read_csv(r'C:\Labmentix\Food waste Management\claims_data.csv')

# Fix date format for Expiry_Date in food_listings_df
food_listings_df['Expiry_Date'] = pd.to_datetime(food_listings_df['Expiry_Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

print("CSV files loaded and date formatted successfully!")

# Optional: Check first few rows of any dataframe to verify
print(providers_df.head())


CSV files loaded and date formatted successfully!
   Provider_ID                         Name           Type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson and Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince and Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             Address            City  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                Contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  
3      556.944.8935x401  
4          193

In [67]:
def insert_data(df, table_name):
    cols = ",".join([str(i) for i in df.columns.tolist()])
    placeholders = ",".join(["%s"] * len(df.columns))
    sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
    
    data = [tuple(x) for x in df.to_numpy()]
    cursor.executemany(sql, data)
    conn.commit()
    print(f"Inserted {cursor.rowcount} rows into {table_name}")

# Flexible parsing without fixed format
food_listings_df['Expiry_Date'] = pd.to_datetime(food_listings_df['Expiry_Date'], errors='coerce').dt.strftime('%Y-%m-%d')
claims_df['Timestamp'] = pd.to_datetime(claims_df['Timestamp'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

# Clear tables first
cursor.execute("DELETE FROM Claims")
cursor.execute("DELETE FROM Food_Listings")
cursor.execute("DELETE FROM Receivers")
cursor.execute("DELETE FROM Providers")
conn.commit()

# Then insert fresh data
insert_data(providers_df, 'Providers')
insert_data(receivers_df, 'Receivers')
insert_data(food_listings_df, 'Food_Listings')
insert_data(claims_df, 'Claims')

Inserted 1000 rows into Providers
Inserted 1000 rows into Receivers
Inserted 1000 rows into Food_Listings
Inserted 1000 rows into Claims


In [20]:
!streamlit run food.py

^C
